# Classical Component VQE

In [1]:
import qsharp
# qsharp.reload() helped when couldn't locate any namespaces
# qsharp.get_available_operations_by_namespace() was good for debugging

In [2]:
qsharp.get_available_operations_by_namespace()

{'SLAP': ['Ansatz', 'GetEnergyByTrotterization', 'VQEGetEnergy']}

In [3]:
packs = qsharp.Packages(qsharp.client)
packs.add(package_name="Microsoft.Quantum.Chemistry")

Adding package Microsoft.Quantum.Chemistry.

In [4]:
qsharp.reload()
qsharp.get_available_operations_by_namespace()

Reloading workspace.

{'SLAP': ['Ansatz', 'GetEnergyByTrotterization', 'VQEGetEnergy']}

In [5]:
from SLAP import VQEGetEnergy, GetEnergyByTrotterization

In [6]:
# from qdk.chemistry import Molecule
from qdk.chemistry.broombridge import load_and_encode

Adding package microsoft.quantum.chemistry.jupyter.

In [7]:
# caffine = Molecule.from_xyz("data/caffeine/caffeine.xyz")

In [8]:
# caffine.mol

In [9]:
encoded_data = load_and_encode("data/HHO.yaml")

In [10]:
from scipy.optimize import minimize

def VQEGetEnergyWrapper(var_params, molecule_data, num_samples) -> float:
    theta1, theta2, theta3 = var_params
    return VQEGetEnergy.simulate(
        JWEncodedData=molecule_data,
        theta1=theta1,
        theta2=theta2,
        theta3=theta3,
        nSamples=num_samples
    )

def VQE_Classical_Routine(molecule_data, initial_var_params, num_samples):
    """ Run VQE Optimization to find the optimal energy and the associated variational parameters """
    opt_result = minimize(
        VQEGetEnergyWrapper,
        initial_var_params,
        args=(molecule_data, num_samples,),
        method="COBYLA",
        tol=0.000001,
        options={'disp': True, 'maxiter': 200,'rhobeg' : 0.05}
    )

    if opt_result.success:
        print(opt_result.message)
        print(f"Lowest Energy: {opt_result.fun} Ha")
        print(f"Number of evaluations: {opt_result.nfev}")
        print(f"Optimal parameters found: {opt_result.x}")

    return opt_result

In [11]:
# VQE_Classical_Routine(encoded_data_caffeine, [0.001, -0.001, 0.001], 10000000)

In [12]:
VQE_Classical_Routine(encoded_data, [0.001, -0.001, 0.001], 100)

Optimization terminated successfully.
Lowest Energy: -73.34845503070379 Ha
Number of evaluations: 41
Optimal parameters found: [-0.03322566 -0.01654004 -0.0348015 ]


     fun: -73.34845503070379
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 41
  status: 1
 success: True
       x: array([-0.03322566, -0.01654004, -0.0348015 ])

In [13]:
VQEGetEnergy.estimate_resources(
    JWEncodedData=encoded_data,
    theta1=0.001,
    theta2=-0.001,
    theta3=0.001,
    nSamples=100
)

{'CNOT': 0,
 'QubitClifford': 1393000,
 'R': 0,
 'Measure': 2089500,
 'T': 0,
 'Depth': 0,
 'Width': 14,
 'QubitCount': 14,
 'BorrowedWidth': 0}

In [14]:
GetEnergyByTrotterization.simulate(
    qSharpData=encoded_data,
    nBitsPrecision=10,
    trotterStepSize=0.2,
    trotterOrder=1
)

In [ ]:
GetEnergyByTrotterization.estimate_resources(
    qSharpData=encoded_data,
    nBitsPrecision=10,
    trotterStepSize=0.2,
    trotterOrder=1
)

{'CNOT': 1736992,
 'QubitClifford': 1022362,
 'R': 664444,
 'Measure': 605,
 'T': 0,
 'Depth': 0,
 'Width': 5,
 'QubitCount': 5,
 'BorrowedWidth': 0}